# Вычислительный граф функции

a^3 - 3a + b + 2ab - x

In [106]:
import math

In [107]:
class Operation():
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.output_nodes = []

        for node in input_nodes:
            node.output_nodes.append(self)

        _default_graph.operations.append(self)
  
    def compute(self):
        pass

### Сложение

In [108]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

### Вычитание

In [109]:
class substract(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var - y_var

### Произведение переменных

In [110]:
class multiply(Operation):
     
    def __init__(self, a, b):
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
        self.inputs = [a_var, b_var]
        return a_var * b_var

### Возведение в степень

In [111]:
class power(Operation):
     
    def __init__(self, a, n):
        super().__init__([a, n])
    
    def compute(self, a_var, n):
        self.inputs = [a_var, n]
        return math.pow(a_var, n)

### Placeholder

In [112]:
class Placeholder():

    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

### Constant

Отдельный тип данных для констант

In [113]:
class Constant():
    
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []

### Variable

In [114]:
class Variable():
    
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []
         
        _default_graph.variables.append(self)

### Graph

In [115]:
class Graph():
        
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [116]:
g = Graph()
g.set_as_default()

In [117]:
a = Variable(10)
b = Variable(3)
n = Constant(3)
m = Constant(2)

In [118]:
x = Placeholder()

In [119]:
op1 = power(a, n)

In [120]:
op2 = multiply(a, n)

In [121]:
op3 = substract(op1, op2)

In [122]:
op4 = add(op3, b)

In [123]:
op5 = multiply(a, b)

In [124]:
op6 = multiply(op5, m)

In [125]:
op7 = add(op4, op6)

In [126]:
z = substract(op7, x)

In [127]:
import numpy as np

In [128]:
def traverse_postorder(operation):

    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [129]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable or type(node) == Constant:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                 
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        return operation.output

In [130]:
sess = Session()

In [131]:
result = sess.run(operation=z,feed_dict={x:33})

In [132]:
result

1000.0

Проверка:

In [133]:
math.pow(10, 3) - 3*10 + 3 + 2*10*3 - 33

1000.0